## Vgg19 Model

### Import packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from keras.applications import vgg19
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint

### Directory

In [ ]:
base_dir =r'D:\datasets'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir, 'test')

### Model

In [ ]:
from keras import models
from keras import layers


conv_base = vgg19.VGG19(weights = 'imagenet', include_top=False, input_shape=(224, 224, 3))

x = conv_base.output
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(.4)(x)


predictions = layers.Dense(5, activation='softmax')(x)
model = models.Model(inputs=conv_base.input, outputs=predictions)

### Image Preprocessing

In [ ]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=30,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                target_size=(224, 224),
                                batch_size=10,
                                class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                target_size=(224, 224),
                                batch_size=10,
                                class_mode='categorical')

In [ ]:
labels=(train_generator.class_indices)
print(labels)

### Compile and Fit

In [ ]:
from keras.metrics import AUC
model.compile(loss='categorical_crossentropy',
                                optimizer=optimizers.Adam(learning_rate=1e-06),
                                metrics=['acc', AUC()])


filepath = "vgg19_final_weights_all.best.2.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only=True, mode = 'max')
callbacks_list = [checkpoint]

history = model.fit_generator(train_generator,
                                steps_per_epoch=100,
                                epochs=5, 
                                callbacks=callbacks_list, 
                                verbose =1, 
                                validation_data=validation_generator,
                                validation_steps=100)

### Evaluate Model 

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                target_size=(224, 224),
                                batch_size=1,
                                class_mode='categorical')
model.evaluate(test_generator)

### Confusion Matrix, Percision, Recall, f-1 score

In [ ]:
y_true = test_generator.classes
predictions = model.predict_generator(test_generator)
y_pred = np.array([np.argmax(x) for x in predictions])
y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_true, y_pred)

In [ ]:
import seaborn as sns

ax = sns.heatmap(cm, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))

### Plot Accuracy Graph and Matrices

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc =history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

auc = history.history['auc_1']
val_auc = history.history['val_auc_1']



epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label= 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()
plt.plot(epochs, auc, 'bo', label= 'AUC')
plt.plot(epochs, val_auc, 'b', label = 'AUC loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### Individual Test

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

validation_dir=r"D:\datasets\validation"
generator = ImageDataGenerator()
validation_ds = generator.flow_from_directory(validation_dir,target_size=(224, 224),batch_size=10)

# model.evaluate_generator(validation_ds)

# Get the names of the 5 classes
class_names = validation_ds.class_indices.keys()

preds = model.predict(validation_ds)
pred_classes = np.argmax(preds, axis=1)

true_classes = validation_ds.classes


def plot_heatmap(y_true, y_pred, class_names, ax, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm, 
        annot=True, 
        square=True, 
        xticklabels=class_names, 
        yticklabels=class_names,
        fmt='d', 
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax
    )
    ax.set_title(title, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)

fig, (ax1) = plt.subplots(1, 1, figsize=(10, 10))

# plot_heatmap(true_classes, scratch_pred_classes, class_names, ax1, title="Custom CNN")    
plot_heatmap(true_classes, pred_classes, class_names, ax1, title="Transfer Learning (VGG16) No Fine-Tuning")    
# plot_heatmap(true_classes, vgg_pred_classes_ft, class_names, ax3, title="Transfer Learning (VGG16) with Fine-Tuning")    

# fig.suptitle("Confusion Matrix Model Comparison", fontsize=24)
fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()